In [277]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
import os
from sklearn.ensemble import RandomForestClassifier


# Trova il percorso assoluto della cartella "esotic" (dove si trova il tuo ipynb)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Trova il percorso assoluto della cartella "lib"
lib_dir = os.path.join(current_dir, "..", "lib")

# Aggiungi il percorso di "lib" al PYTHONPATH
sys.path.append(lib_dir)

from Utility import *

features = [
    'EloDiff', 'feat1', 'FormDiff'
]

features2 = ['UltimoScontroDiretto', 'feat1', 'feat2']

## DATA

In [278]:
data = getAltriData().copy()
# data = pd.read_excel('../data/mergedSerieA.xlsx')

dataU = data.dropna(subset=features2)


## Feature Engineering

In [279]:
data = data.dropna(subset=features)
print(len(data), len(dataU))
# data = data.dropna()
print(len(data))
data['isOver'] = np.where(data['MatchGoal'] > 2.5, 1, 0)
print('Over:', round(100*data[data['isOver']==1]['isOver'].count() / len(data['isOver'])), '%')
x_train, x_test, y_train, y_test = train_test_split(
    data[features], data['isOver'], test_size=0.98, random_state=42, shuffle=True)

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()


print(f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

12975 7783
12975
Over: 46 %
X_train: (259, 3) 
X_test: (12716, 3) 
y_train: (259,) 
y_test: (12716,)


In [280]:

# data = data.dropna()
print(len(dataU))
dataU['isOver'] = np.where(dataU['MatchGoal'] > 2.5, 1, 0)
print('Over:', round(100*dataU[dataU['isOver']==1]['isOver'].count() / len(dataU['isOver'])), '%')
x_train_u, x_test_u, y_train_u, y_test_u = train_test_split(
    dataU[features2], dataU['isOver'], test_size=0.98, random_state=42, shuffle=True)

x_train_u = x_train_u.sort_index()
x_test_u = x_test_u.sort_index()
y_train_u = y_train_u.sort_index()
y_test_u = y_test_u.sort_index()

7783
Over: 46 %


## Models

### Decision Tree model

In [281]:
from sklearn.tree import DecisionTreeClassifier



# Crea e addestra il modello di albero di decisione per la regressione
decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
                                 # , class_weight={1: 1.2, 0: 1}
                                 # , class_weight='balanced'
                                 , max_depth=3  # , ccp_alpha= 0.00001
                                 # , max_features= 3
                                 ).fit(x_train, y_train)

# decTreeModel = RandomForestClassifier(random_state=42, n_estimators=100, 
#                                 min_samples_split=80, max_depth=3).fit(x_train_under, y_train_under) 

# dumpModel("Step1.joblib", decTreeModel)
# decTreeModel = loadModel("Step1.joblib")

### LogisticRegression

In [282]:
logRegModel = trainLogRegModel(x_train, y_train)

logRegModel_u = trainLogRegModel(x_train_u, y_train_u)
# dumpModel("Step1LogReg.joblib", logRegModel_u)
rfModel = loadModel('Step1LogReg.joblib')

### Random Forest

In [283]:
# Crea e addestra il modello di Random Forest


from sklearn.model_selection import GridSearchCV

rfModel = RandomForestClassifier(random_state=42, n_estimators=200, 
                                min_samples_split=20, max_depth=3,
                                # class_weight='balanced_subsample'
                                ).fit(x_train, y_train) 


# param_grid = {
#     'n_estimators': [200],
#     'min_samples_split': [50, 100],
#     'max_depth': [3, 5],
# }

# grid_search = GridSearchCV(estimator=rfModel, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train, y_train)

# print("Migliori iperparametri:", grid_search.best_params_)
# print("Migliore accuratezza:", grid_search.best_score_)

# best_rfModel = grid_search.best_estimator_

# rfModel = best_rfModel


# dumpModel("Step1RF.joblib", rfModel)
rfModel = loadModel('Step1RF.joblib')

In [284]:
from sklearn.multiclass import OneVsRestClassifier

# Crea un classificatore One-vs-Rest usando Logistic Regression come classificatore base
ovr_classifier = OneVsRestClassifier(LogisticRegression())

# Addestra il classificatore
ovr_classifier.fit(x_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression())

## Utility Func for Gain

In [285]:
quotaMin = 1.50
def calculate_gain_O25(row, prediction):
    
    if (row[prediction] == 1):
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    elif (row[prediction] == 0): 
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_Over25(row, prediction):
    # print('row', row['B365>2.5'], pd.isna(row['B365>2.5']))
    if (row[prediction] == 1):
        # return 0
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0
    
def calculate_gain_Under25(row, prediction):
    if (row[prediction] == 0): 
        # return 0
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

## Metrics

In [286]:
from itertools import count
from math import nan
from sklearn.metrics import accuracy_score

df_train = data[data.index.isin(x_train.index)].copy()
print(len(df_train))

df_train['LogRegPred'] = logRegModel.predict(x_train)
df_train['DecTreePred'] = decTreeModel.predict(x_train)
df_train['RFPred'] = rfModel.predict(x_train)


df_test = data[data.index.isin(x_test.index)].copy()
df_test_u = dataU[dataU.index.isin(x_test_u.index)].copy()
print(len(df_test))

df_test['LogRegPred'] = logRegModel.predict(x_test)
df_test['DecTreePred'] = decTreeModel.predict(x_test)
df_test['RFPred'] = rfModel.predict(x_test)
df_test['1vsAllPred'] = ovr_classifier.predict(x_test)
df_test['FullPred'] = np.where(df_test['RFPred']+df_test['LogRegPred'] ==2, 1, 0)
df_test_u['LogRegPred'] = logRegModel_u.predict(x_test_u)
print(
    f"Test LogReg Accuracy score: {round(100*accuracy_score(y_test, df_test['LogRegPred']),2)}%")
print(
    f"Test DecTree Accuracy score: {round(100*accuracy_score(y_test, df_test['DecTreePred']),2)}%")
print(
    f"Train Random Forest Accuracy score: {round(100*accuracy_score(y_train, df_train['RFPred']),2)}%")
print(
    f"Test Random Forest Accuracy score: {round(100*accuracy_score(y_test, df_test['RFPred']),2)}%")
print(
    f"Test OneVsAll Accuracy score: {round(100*accuracy_score(y_test, df_test['1vsAllPred']),2)}%")
print(
    f"Test FullPred Accuracy score: {round(100*accuracy_score(y_test, df_test['FullPred']),2)}%")
print(
    f"Test Log Reg Accuracy score Under: {round(100*accuracy_score(y_test_u, df_test_u['LogRegPred']),2)}%")

models = ['LogReg', 'RF', 'DecTree', '1vsAll']
rows = []
classAccuracyDf = pd.DataFrame(
    columns=['Modello', 'Accuratezza_Under', 'Accuratezza_Over'])
for model in models:
    row = {
        "Modello": model,
        "Accuratezza_Over": class_accuracy(y_test, df_test[model+'Pred'])['Over'],
        "Accuratezza_Under": class_accuracy(y_test, df_test[model+'Pred'])['Under']
    }
    rows.append(row)

classAccuracyDf = pd.DataFrame(rows)
classAccuracyDf.set_index('Modello', inplace=True)

print('')
print(classAccuracyDf)


prediction = 'DecTreePred'
prediction = 'RFPred'
# prediction = 'LogRegPred'
predictionU = 'LogRegPred'
# df_test['B365<2.5'] = 1.9
df_test['B365<2.5'] = np.where(pd.isna(df_test['B365<2.5']), 1.9, df_test['B365<2.5'])
df_test['B365>2.5'] = np.where(pd.isna(df_test['B365>2.5']), 1.9, df_test['B365>2.5'])
df_test_u['B365<2.5'] = np.where(pd.isna(df_test_u['B365<2.5']), 1.9, df_test_u['B365<2.5'])
df_test_u['B365>2.5'] = np.where(pd.isna(df_test_u['B365>2.5']), 1.9, df_test_u['B365>2.5'])

df_test['GainOver'] = df_test.apply(calculate_gain_Over25, axis=1, args=(prediction,) )
df_test_u['GainUnder'] = df_test_u.apply(calculate_gain_Under25, axis=1, args=(predictionU,) )
# df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
# df_test['Equity'] = df_test['Gain'].cumsum()
df_test['EquityOver'] = df_test['GainOver'].cumsum()
df_test_u['EquityUnder'] = df_test_u['GainUnder'].cumsum()

# print(df_test['Equity'].tail(1))
print(df_test['EquityOver'].tail(1))
print(df_test_u['EquityUnder'].tail(1))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['isOver'], df_test[prediction])
print(cm)

train_to_export = df_train[df_train[prediction] ==1].copy()
print(len(train_to_export))
# train_to_export.to_excel("../Dataframe/Train Over Step1.xlsx")
test_to_export = df_test[df_test[prediction] ==1].copy()
print(len(test_to_export))
# test_to_export.to_excel("../Dataframe/Test Over Step1.xlsx")

# exportExcelWithTimeStamp(df_test, "../Dataframe/", "df.xlsx")

259
12716
Test LogReg Accuracy score: 51.26%
Test DecTree Accuracy score: 53.79%
Train Random Forest Accuracy score: 50.97%
Test Random Forest Accuracy score: 53.81%
Test OneVsAll Accuracy score: 51.23%
Test FullPred Accuracy score: 54.07%
Test Log Reg Accuracy score Under: 50.24%

         Accuratezza_Over  Accuratezza_Under
Modello                                     
LogReg           0.464674           0.544173
RF               0.482830           0.542578
DecTree          0.488168           0.544451
1vsAll           0.464236           0.543878
16497   -77.39
Name: EquityOver, dtype: float64
16486    7.8
Name: EquityUnder, dtype: float64
[[6378  497]
 [5377  464]]
16
961


## Second Round

In [287]:
dataTrain = train_to_export#getTrainOver1Data().copy()
dataTest = test_to_export#getTestOver1Data().copy()
dataTrain = dataTrain.dropna(subset=features2)
dataTest = dataTest.dropna(subset=features2)
# data2 = df_merged[df_merged[prediction] ==1].copy()


# data = data.dropna()

x_train = dataTrain[features2]
y_train = dataTrain['isOver']
x_test = dataTest[features2]
y_test = dataTest['isOver']

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()

print(f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

X_train: (9, 3) 
X_test: (482, 3) 
y_train: (9,) 
y_test: (482,)


In [288]:
# Crea e addestra il modello di albero di decisione per la regressione
decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
                                 # , class_weight={1: 1.2, 0: 1}
                                 # , class_weight='balanced'
                                 , max_depth=3  # , ccp_alpha= 0.00001
                                 # , max_features= 3
                                 ).fit(x_train, y_train)

# decTreeModel = RandomForestClassifier(random_state=42, n_estimators=100, 
#                                 min_samples_split=80, max_depth=3).fit(x_train_under, y_train_under) 

# dumpModel("Under.joblib", decTreeModel)
# decTreeModel = loadModel("Under.joblib")
logRegModel = trainLogRegModel(x_train, y_train)
# dumpModel("Step2.joblib", decTreeModel)
# logRegModel = loadModel("Step2.joblib")

# Crea e addestra il modello di Random Forest
rfModel2 = RandomForestClassifier(random_state=42, n_estimators=100, 
                                min_samples_split=80, max_depth=3).fit(x_train, y_train) 

# dumpModel("Over.joblib", rfModel2)
# rfModel2 = loadModel('Over.joblib')

In [289]:
from itertools import count
from sklearn.metrics import accuracy_score
df_test = dataTest[dataTest.index.isin(x_test.index)].copy()
print(len(df_test))

df_test['LogRegPred'] = logRegModel.predict(x_test)
df_test['DecTreePred'] = decTreeModel.predict(x_test)
df_test['RFPred'] = rfModel2.predict(x_test)



print(
    f"Test LogReg Accuracy score: {round(100*accuracy_score(y_test, df_test['LogRegPred']),2)}%")
print(
    f"Test DecTree Accuracy score: {round(100*accuracy_score(y_test, df_test['DecTreePred']),2)}%")
print(
    f"Test Random Forest Accuracy score: {round(100*accuracy_score(y_test, df_test['RFPred']),2)}%")

models = ['LogReg', 'RF', 'DecTree']
rows = []
classAccuracyDf = pd.DataFrame(
    columns=['Modello', 'Accuratezza_Under', 'Accuratezza_Over'])
for model in models:
    row = {
        "Modello": model,
        "Accuratezza_Over": class_accuracy(y_test, df_test[model+'Pred'])['Over'],
        "Accuratezza_Under": class_accuracy(y_test, df_test[model+'Pred'])['Under']
    }
    rows.append(row)

classAccuracyDf = pd.DataFrame(rows)
classAccuracyDf.set_index('Modello', inplace=True)

print('')
print(classAccuracyDf)


prediction = 'DecTreePred'
# prediction = 'RFPred'
# prediction = 'LogRegPred'

df_test['B365<2.5'] = 1.8
df_test['B365>2.5'] = 1.8


df_test['GainOver'] = df_test.apply(calculate_gain_Over25, axis=1, args=(prediction,) )
df_test['GainUnder'] = df_test.apply(calculate_gain_Under25, axis=1, args=(prediction,) )
# df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
# df_test['Equity'] = df_test['Gain'].cumsum()
df_test['EquityOver'] = df_test['GainOver'].cumsum()
df_test['EquityUnder'] = df_test['GainUnder'].cumsum()

# print(df_test['Equity'].tail(1))
print(df_test['EquityOver'].tail(1))
print(df_test['EquityUnder'].tail(1))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['isOver'], df_test[prediction])
print(cm)
# exportExcelWithTimeStamp(df_test_under, "../Dataframe/", "df.xlsx")



482
Test LogReg Accuracy score: 48.96%
Test DecTree Accuracy score: 50.41%
Test Random Forest Accuracy score: 50.41%

         Accuratezza_Over  Accuratezza_Under
Modello                                     
LogReg           0.480663           0.495017
RF                    NaN           0.504149
DecTree               NaN           0.504149
16473    0
Name: EquityOver, dtype: int64
16473   -44.6
Name: EquityUnder, dtype: float64
[[243   0]
 [239   0]]


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\Esotic\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\Esotic\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\Esotic\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\Esotic\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
